In [78]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

#試取資料
file = open("/Users/peggy/Documents/109-2(2-2)/machine_learning/MLGame/games/snake/log/snake1.pickle", "rb")
data = pickle.load(file)
file.close()
type(data['ml'])

dict

In [79]:
game_info = data["ml"]["scene_info"]
game_command = data["ml"]["command"]
#print(game_info)
#print(game_command)

In [80]:

for i in range(2,3): 
    path = "/Users/peggy/Documents/109-2(2-2)/machine_learning/MLGame/games/snake/log/snake" + str(i) + ".pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml']['scene_info']
    game_command = game_command + data['ml']['command']
    file.close()  

print(len(game_info))
print(len(game_command))


158034
158034


In [81]:
g = game_info[1]

feature = np.array([g["snake_head"][0], g["snake_head"][1]])
print(feature)
print(game_command[1])
if game_command[1] == "UP": game_command[1] = 0
elif game_command[1] == "DOWN": game_command[1] = 1
elif game_command[1] == "LEFT": game_command[1] = 2
elif game_command[1] == "RIGHT": game_command[1] = 3
else: game_command[1] = 4 

[40 50]
LEFT


In [82]:
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    
    feature = np.vstack((feature, [g["snake_head"][0], g["snake_head"][1]]))
    if game_command[i] == "UP": game_command[i] = 0
    elif game_command[i] == "DOWN": game_command[i] = 1
    elif game_command[i] == "LEFT": game_command[i] = 2
    elif game_command[i] == "RIGHT": game_command[i] = 3
    else: game_command[i] = 4 
        
answer = np.array(game_command[1:-1])
print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 40  50]
 [ 30  50]
 [ 20  50]
 ...
 [270   0]
 [280   0]
 [290   0]]
(158032, 2)
[2 2 2 ... 3 3 3]
(158032,)


# KNN


In [83]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state=9)
#參數區間
param_grid = {'n_neighbors':[1, 2, 3, 4, 5]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(n_neighbors = 5), param_grid, cv=cv, verbose=10, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('model_KNN.pickle', 'wb')
pickle.dump(grid, file)
file.close()


Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    3.7s remaining:    8.6s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    3.7s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    3.7s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.6s finished


In [278]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 5}
[[6164   75  373  345    0]
 [  71 6402  356  352    0]
 [ 351  360 8975  105    0]
 [ 368  397  123 8683    0]
 [   5    5    7   25    1]]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      6957
           1       0.88      0.89      0.89      7181
           2       0.91      0.92      0.91      9791
           3       0.91      0.91      0.91      9571
           4       1.00      0.02      0.05        43

    accuracy                           0.90     33543
   macro avg       0.92      0.72      0.73     33543
weighted avg       0.90      0.90      0.90     33543

